In [1]:
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup

In [2]:
dfs = pd.read_html("https://fbref.com/en/squads/18bb7c10/Arsenal-Stats")

In [3]:
pl_2022_2023_links = {
    "Arsenal": "https://fbref.com/en/squads/18bb7c10/Arsenal-Stats",
    "Manchester_City": "https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats",
    "Manchester_United": "https://fbref.com/en/squads/19538871/Manchester-United-Stats",
    "Newcastle_United": "https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats",
    "Tottenham": "https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats",
    "Brighton": "https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats",
    "Brentford": "https://fbref.com/en/squads/cd051869/Brentford-Stats",
    "Fulham": "https://fbref.com/en/squads/fd962109/Fulham-Stats",
    "Chelsea": "https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats",
    "Liverpool": "https://fbref.com/en/squads/822bd0ba/Liverpool-Stats",
    "Aston_Villa": "https://fbref.com/en/squads/8602292d/Aston-Villa-Stats",
    "Crystal_Palance": "https://fbref.com/en/squads/47c64c55/Crystal-Palace-Stats",
    "Forest": "https://fbref.com/en/squads/e4a775cb/Nottingham-Forest-Stats",
    "Leicester": "https://fbref.com/en/squads/a2d435b3/Leicester-City-Stats",
    "Wolves": "https://fbref.com/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats",
    "West_Ham": "https://fbref.com/en/squads/7c21e445/West-Ham-United-Stats",
    "Leeds": "https://fbref.com/en/squads/5bfb9659/Leeds-United-Stats",
    "Everton": "https://fbref.com/en/squads/d3fd31cc/Everton-Stats",
    "Bournemouth": "https://fbref.com/en/squads/4ba7cbea/Bournemouth-Stats",
    "Southampton": "https://fbref.com/en/squads/33c895d4/Southampton-Stats",
}

df_order = [
    "overall",
    "fixtures",
    "gk",
    "advanced_gk",
    "shooting",
    "passing",
    "pass_types",
    "goal_and_shot_creation",
    "defensive_actions",
    "possession",
    "playing_time",
    "miscellaneous_tasks",
    "overall_regular_season",
    "home_away_regular_season",
]



In [167]:
for club in pl_2022_2023_links:
    current_club = club
    df_list = pd.read_html(pl_2022_2023_links[current_club])
    for i, df in enumerate(df_list):
        if not os.path.isdir(f"pl_data/{current_club}/"):
            os.makedirs(f"pl_data/{current_club}/")
        df.to_csv(f"pl_data/{current_club}/{df_order[i]}.csv", index=False)

### Acquiring specific team data

In [152]:
headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

page = "https://www.transfermarkt.us/fulham-fc/kader/verein/931/saison_id/2022/plus/1"
pageTree = requests.get(page, headers=headers)
pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

In [174]:
names = []
positions = []
tables= pageSoup.find_all('table', class_='inline-table')
for table in tables:
    rows = table.find_all('tr')
    name = rows[0].text.strip()
    position = rows[1].text.strip()
    names.append(name)
    positions.append(position)

In [154]:
player_prices = pageSoup.find_all(class_='rechts hauptlink')
prices = []
for price in player_prices:
    price = price.text.strip()
    prices.append(price)

In [175]:
info = pd.DataFrame({'name': names, 'price': prices, 'position':positions})

In [158]:
report = []

tables= pageSoup.find_all('table', class_='items')
for table in tables:
    rows = table.find_all('td', {"class": "zentriert"})
    for i in range(len(rows)):
        report.append(rows[i])

general = [report[i:i + 8] for i in range(0, len(report), 8)]
overall = []
for g in general:
     overall.append(dict(
         dob = g[1].text.strip(),
         nationality = g[2].img['alt'],
         height = g[3].text.strip(),
         foot = g[4].text.strip(),
         join_date = g[5].text.strip(),
         contract_expiration = g[7].text.strip(), 
     )
    )

In [160]:
info2 = pd.DataFrame(overall)

In [281]:
overall = pd.concat([info, info2], axis=1)

In [282]:
overall

,name,price,position,dob,nationality,height,foot,join_date,contract_expiration
0,Bernd Leno,€10.00m,Goalkeeper,"Mar 4, 1992 (30)",Germany,"1,90m",right,"Aug 2, 2022","Jun 30, 2025"
1,Marek Rodak,€3.50m,Goalkeeper,"Dec 13, 1996 (26)",Slovakia,"1,94m",right,"Jul 1, 2019","Jun 30, 2024"
2,Tosin Adarabioyo,€18.00m,Centre-Back,"Sep 24, 1997 (25)",England,"1,96m",right,"Oct 5, 2020","Jun 30, 2024"
3,Issa Diop,€15.00m,Centre-Back,"Jan 9, 1997 (26)",France,"1,94m",right,"Aug 10, 2022","Jun 30, 2027"
4,Shane Duffy,€2.50m,Centre-Back,"Jan 1, 1992 (31)",Ireland,"1,93m",right,"Jan 31, 2023","Jun 30, 2023"
5,Tim Ream,€1.00m,Centre-Back,"Oct 5, 1987 (35)",United States,"1,86m",left,"Aug 20, 2015","Jun 30, 2024"
6,Antonee Robinson,€10.00m,Left-Back,"Aug 8, 1997 (25)",United States,"1,83m",left,"Aug 20, 2020","Jun 30, 2024"
7,Layvin Kurzawa,€5.00m,Left-Back,"Sep 4, 1992 (30)",France,"1,82m",left,"Sep 1, 2022","Jun 30, 2023"
8,Kenny Tete,€7.00m,Right-Back,"Oct 9, 1995 (27)",Netherlands,"1,80m",right,"Sep 10, 2020","Jun 30, 2024"
9,Kevin Mbabu,€7.00m,Right-Back,"Apr 19, 1995 (27)",Switzerland,"1,84m",right,"Jul 27, 2022","Jun 30, 2025"


## Advanced GK

In [247]:
advanced_gk = pd.read_csv("pl_data/Fulham/advanced_gk.csv", header=1)

In [248]:
advanced_gk = advanced_gk.iloc[:-2]

In [249]:
advanced_gk

,Player,Nation,Pos,Age,90s,GA,PKA,FK,CK,OG,...,Att.2,Launch%.1,AvgLen.1,Opp,Stp,Stp%,#OPA,#OPA/90,AvgDist,Matches
0,Bernd Leno,de GER,GK,30-342,20.0,28,3,1,4,0,...,139,41.7,37.1,256,22,8.6,27,1.35,14.7,Matches
1,Marek Rodák,sk SVK,GK,26-058,2.0,2,0,0,0,0,...,15,86.7,60.0,26,1,3.8,2,1.00,14.0,Matches


Player
Nation
Position
Age
90s
GA - goals against
pka - penalty kicks allowed
fk - freekick goals allowed
ck - corner kick goals allowed
og - own goals allowed
PSxG - expected goals based on how likely the goalkeeper is expected to save the shot
PSxG/SoT - expected goals per shot on target
PSxG+/- - expected goals minus allowed goals
/90 - expected goals minus allowed goals per 90 minutes
Cmp - passes completed over 40 yards
Att - passes attemtped over 40 yards
Cmp% - completion ratio for passes over 40 yards


## Defensive Actions 

In [250]:
defensive_actions = pd.read_csv("pl_data/Fulham/defensive_actions.csv", header=1)

In [253]:
defensive_actions.tail()

,Player,Nation,Pos,Age,90s,Tkl,TklW,Def 3rd,Mid 3rd,Att 3rd,...,Tkl%,Lost,Blocks,Sh,Pass,Int,Tkl+Int,Clr,Err,Matches
22,Manor Solomon,il ISR,FW,23-200,0.7,2,1,1,1,0,...,100.0,0,0,0,0,1,3,0,0,Matches
23,Josh Onomah,eng ENG,MF,25-288,0.2,1,0,1,0,0,...,NaN,0,1,0,1,0,1,0,0,Matches
24,Luke Harris,wls WAL,"MF,FW",17-312,0.1,0,0,0,0,0,...,NaN,0,0,0,0,0,0,0,0,Matches
25,Shane Duffy,ie IRL,"FW,DF",31-039,0.1,0,0,0,0,0,...,NaN,0,0,0,0,0,0,0,0,Matches
26,Tyrese Francois,au AUS,FW,22-208,0.0,0,0,0,0,0,...,NaN,0,0,0,0,0,0,1,0,Matches


In [252]:
defensive_actions = defensive_actions.iloc[:-2]

## GK 

In [254]:
gk = pd.read_csv("pl_data/Fulham/gk.csv", header=1)

In [258]:
gk.head()

,Player,Nation,Pos,Age,MP,Starts,Min,90s,GA,GA90,...,D,L,CS,CS%,PKatt,PKA,PKsv,PKm,Save%.1,Matches


In [257]:
gk = gk.iloc[:-2]

## Goal and Shot Creation

In [259]:
goal_and_shot = pd.read_csv("pl_data/Fulham/goal_and_shot_creation.csv", header=1)

goal_and_shot.head()

In [260]:
goal_and_shot = goal_and_shot.iloc[:-2]

## Misc

In [261]:
misc = pd.read_csv("pl_data/Fulham/miscellaneous_tasks.csv", header=1)

In [262]:
misc = misc.iloc[:-2]

In [219]:
misc.head()

,Player,Nation,Pos,Age,90s,CrdY,CrdR,2CrdY,Fls,Fld,...,Int,TklW,PKwon,PKcon,OG,Recov,Won,Lost,Won%,Matches
0,Tim Ream,us USA,DF,35-127,22.0,2,0,0,11,5,...,32,11,0,0,0,131,29,18,61.7,Matches
1,Andreas Pereira,br BRA,MF,27-039,20.2,6,0,0,28,23,...,4,6,0,1,0,84,7,19,26.9,Matches
2,João Palhinha,pt POR,MF,27-215,20.7,8,0,0,25,26,...,27,56,0,0,0,125,42,25,62.7,Matches
3,Bobby Reid,jm JAM,"FW,DF",30-007,19.3,7,0,0,29,14,...,13,19,2,1,0,99,9,26,25.7,Matches
4,Harrison Reed,eng ENG,MF,28-013,19.2,5,0,0,19,13,...,13,15,0,0,0,96,19,22,46.3,Matches


## Passing

In [263]:
passing = pd.read_csv("pl_data/Fulham/passing.csv", header=1)

In [226]:
passing.head()

,Player,Nation,Pos,Age,90s,Cmp,Att,Cmp%,TotDist,PrgDist,...,Ast,xAG,xA,A-xAG,KP,1/3,PPA,CrsPA,PrgP,Matches
0,Tim Ream,us USA,DF,35-127,22.0,1073,1247,86.0,23344,8058,...,0,0.0,0.2,0.0,0,81,0,0,75,Matches
1,Andreas Pereira,br BRA,MF,27-039,20.2,464,722,64.3,8608,3003,...,5,4.3,3.1,0.7,48,56,17,9,76,Matches
2,João Palhinha,pt POR,MF,27-215,20.7,592,729,81.2,10353,2564,...,0,0.3,0.2,-0.3,3,74,2,0,63,Matches
3,Bobby Reid,jm JAM,"FW,DF",30-007,19.3,477,687,69.4,7239,2503,...,1,1.2,0.9,-0.2,13,27,23,7,43,Matches
4,Harrison Reed,eng ENG,MF,28-013,19.2,550,726,75.8,8410,2184,...,1,1.8,1.0,-0.8,20,48,18,7,56,Matches


In [264]:
passing = passing.iloc[:-2]

## Playing Time

In [265]:
playing_time = pd.read_csv("pl_data/Fulham/playing_time.csv", header=1)

In [266]:
playing_time = playing_time.iloc[:-2]

In [238]:
playing_time.head()

,Player,Nation,Pos,Age,MP,Min,Mn/MP,Min%,90s,Starts,...,onG,onGA,+/-,+/-90,On-Off,onxG,onxGA,xG+/-,xG+/-90,On-Off.1
0,Tim Ream,us USA,DF,35-127,22,1977.0,90.0,99.8,22.0,22,...,32.0,30.0,2.0,0.09,0.09,29.0,36.8,-7.8,-0.35,-3.19
1,Andreas Pereira,br BRA,MF,27-039,22,1815.0,83.0,91.7,20.2,22,...,31.0,25.0,6.0,0.30,2.48,26.7,30.8,-4.2,-0.21,1.72
2,João Palhinha,pt POR,MF,27-215,21,1865.0,89.0,94.2,20.7,21,...,30.0,26.0,4.0,0.19,1.76,27.3,33.3,-6.1,-0.29,0.96
3,Bobby Reid,jm JAM,"FW,DF",30-007,21,1733.0,83.0,87.5,19.3,21,...,28.0,27.0,1.0,0.05,-0.31,24.6,32.2,-7.6,-0.39,-0.37
4,Harrison Reed,eng ENG,MF,28-013,21,1725.0,82.0,87.1,19.2,21,...,27.0,27.0,0.0,0.00,-0.71,24.6,32.7,-8.2,-0.43,-0.60


In [237]:
playing_time = playing_time.drop(columns = ['Matches'])

## Possession

In [267]:
possession = pd.read_csv("pl_data/Fulham/possession.csv", header=1)

In [269]:
possession.head()

,Player,Nation,Pos,Age,90s,Touches,Def Pen,Def 3rd,Mid 3rd,Att 3rd,...,TotDist,PrgDist,PrgC,1/3,CPA,Mis,Dis,Rec,PrgR,Matches
0,Tim Ream,us USA,DF,35-127,22.0,1437,204,748,648,49,...,4842,3277,18,8,0,12,3,938,1,Matches
1,Andreas Pereira,br BRA,MF,27-039,20.2,887,12,68,410,417,...,2644,1187,37,30,8,39,33,562,90,Matches
2,João Palhinha,pt POR,MF,27-215,20.7,1016,50,241,629,157,...,1630,541,4,8,2,21,16,521,14,Matches
3,Bobby Reid,jm JAM,"FW,DF",30-007,19.3,903,46,186,362,366,...,2255,1105,40,28,11,41,23,557,127,Matches
4,Harrison Reed,eng ENG,MF,28-013,19.2,920,35,163,505,259,...,1908,676,19,16,1,29,14,532,50,Matches


In [268]:
possession = possession.iloc[:-2]

## Shooting

In [270]:
shooting = pd.read_csv("pl_data/Fulham/shooting.csv", header=1)

In [244]:
shooting.head()

,Player,Nation,Pos,Age,90s,Gls,Sh,SoT,SoT%,Sh/90,...,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,Matches
0,Tim Ream,us USA,DF,35-127,22.0,1,3,2,66.7,0.14,...,9.8,0,0,0,0.3,0.3,0.09,0.7,0.7,Matches
1,Andreas Pereira,br BRA,MF,27-039,20.2,2,28,11,39.3,1.39,...,21.9,8,1,1,3.5,2.7,0.10,-1.5,-1.7,Matches
2,João Palhinha,pt POR,MF,27-215,20.7,3,27,7,25.9,1.30,...,16.7,0,0,0,2.0,2.0,0.07,1.0,1.0,Matches
3,Bobby Reid,jm JAM,"FW,DF",30-007,19.3,4,23,9,39.1,1.19,...,12.8,0,0,0,3.4,3.4,0.15,0.6,0.6,Matches
4,Harrison Reed,eng ENG,MF,28-013,19.2,2,15,4,26.7,0.78,...,20.3,0,0,0,1.3,1.3,0.09,0.7,0.7,Matches


In [271]:
shooting = shooting.iloc[:-2]

In [272]:
dfs = [
    advanced_gk,
    defensive_actions,
    gk,
    goal_and_shot,
    misc,
    passing,
    playing_time,
    possession,
    shooting,
]

In [278]:
from functools import reduce

df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Player'],
                                            how='outer'), dfs)

In [279]:
df_merged

,Player,Nation_x,Pos_x,Age_x,90s_x,GA_x,PKA_x,FK_x,CK,OG_x,...,Dist,FK_y,PK,PKatt_y,xG,npxG,npxG/Sh,G-xG,np:G-xG,Matches
0,Bernd Leno,de GER,GK,30-342,20.0,28.0,3.0,1.0,4.0,0.0,...,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,Matches
1,Marek Rodák,sk SVK,GK,26-058,2.0,2.0,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,Matches
2,Tim Ream,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.8,0.0,0.0,0.0,0.3,0.3,0.09,0.7,0.7,Matches
3,Andreas Pereira,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,21.9,8.0,1.0,1.0,3.5,2.7,0.10,-1.5,-1.7,Matches
4,João Palhinha,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,16.7,0.0,0.0,0.0,2.0,2.0,0.07,1.0,1.0,Matches
5,Bobby Reid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12.8,0.0,0.0,0.0,3.4,3.4,0.15,0.6,0.6,Matches
6,Harrison Reed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,20.3,0.0,0.0,0.0,1.3,1.3,0.09,0.7,0.7,Matches
7,Antonee Robinson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,27.8,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.0,Matches
8,Aleksandar Mitrović,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15.6,0.0,3.0,6.0,11.3,6.6,0.10,-0.3,1.4,Matches
9,Kenny Tete,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,19.9,0.0,0.0,0.0,0.3,0.3,0.05,-0.3,-0.3,Matches
